In [49]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.connect('exercises.sqlite')
c = conn.cursor()

In [7]:
with open('Members.sql') as file:
    c.executescript(file.read())

In [10]:
with open('Facilities.sql') as file:
    c.executescript(file.read())

In [11]:
with open('Bookings.sql') as file:
    c.executescript(file.read())

In [50]:
#Q1: Some of the facilities charge a fee to members, but some do not.
#Please list the names of the facilities that do.

#c.execute("SELECT name, membercost FROM Facilities WHERE membercost != 0")
#c.fetchall()
pd.read_sql_query("SELECT name, membercost FROM Facilities WHERE membercost != 0", conn)

,name,membercost
0,Tennis Court 1,5.0
1,Tennis Court 2,5.0
2,Massage Room 1,9.9
3,Massage Room 2,9.9
4,Squash Court,3.5


In [51]:
#Q2: How many facilities do not charge a fee to members?

pd.read_sql_query("SELECT COUNT(name) AS free_facility_count FROM Facilities WHERE membercost = 0", conn)

,free_facility_count
0,4


In [52]:
'''Q3: How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. '''

pd.read_sql_query("SELECT facid, name, membercost, monthlymaintenance "
          "FROM Facilities WHERE membercost > 0 AND membercost < 0.2 * monthlymaintenance", conn)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


In [53]:
#Q4: How can you retrieve the details of facilities with ID 1 and 5?
#Write the query without using the OR operator.

pd.read_sql_query("SELECT * FROM Facilities "
          "WHERE facid IN (1,5)", conn)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


In [54]:
'''Q5: How can you produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100? Return the name and monthly maintenance of the facilities
in question.'''

pd.read_sql_query("SELECT name, monthlymaintenance, "
          "CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS maint_cost_description "
          "FROM Facilities", conn)

,name,monthlymaintenance,maint_cost_description
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


In [55]:
#Q6: You'd like to get the first and last name of the last member(s)
#who signed up. Do not use the LIMIT clause for your solution.

pd.read_sql_query("SELECT firstname, surname, joindate FROM Members ORDER BY joindate DESC", conn)

,firstname,surname,joindate
0,Darren,Smith,2012-09-26 18:08:45
1,Erica,Crumpet,2012-09-22 08:36:38
2,John,Hunt,2012-09-19 11:32:45
3,Hyacinth,Tupperware,2012-09-18 19:32:05
4,Millicent,Purview,2012-09-18 19:04:01
5,Henry,Worthington-Smyth,2012-09-17 12:27:15
6,David,Farrell,2012-09-15 08:22:05
7,Henrietta,Rumney,2012-09-05 08:42:35
8,Douglas,Jones,2012-09-02 18:43:05
9,Ramnaresh,Sarwin,2012-09-01 08:44:42


In [56]:
'''Q7: How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.'''

pd.read_sql_query("SELECT DISTINCT (M.surname||M.firstname) AS full_name, F.name "
            "FROM Bookings B "
            "INNER JOIN Facilities F ON F.facid = B.facid "
            "INNER JOIN Members M ON M.memid = B.memid "
            "WHERE F.name LIKE 'Tennis Court%' AND B.memid != 0 "
            "ORDER BY full_name", conn)
# Used 'SELECT DISTINCT CONCAT(M.surname, ', ', M.firstname)...' for MYSQL

,full_name,name
0,BaderFlorence,Tennis Court 2
1,BaderFlorence,Tennis Court 1
2,BakerAnne,Tennis Court 1
3,BakerAnne,Tennis Court 2
4,BakerTimothy,Tennis Court 2
5,BakerTimothy,Tennis Court 1
6,BootheTim,Tennis Court 2
7,BootheTim,Tennis Court 1
8,ButtersGerald,Tennis Court 1
9,ButtersGerald,Tennis Court 2


In [57]:
'''Q8: How can you produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30? Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.'''

pd.read_sql_query("SELECT F.name, (M.surname||M.firstname) AS full_name, "
            "CASE WHEN M.memid = 0 THEN (F.guestcost * B.slots) ELSE (F.membercost * B.slots) END AS booking_cost "
            "FROM Bookings B "
            "INNER JOIN Facilities F ON F.facid = B.facid "
            "INNER JOIN Members M ON M.memid = B.memid "
            "WHERE B.starttime LIKE '2012-09-14%' AND "
            "CASE WHEN M.memid = 0 THEN (F.guestcost * B.slots) ELSE (F.membercost * B.slots) END > 30 "
            "ORDER BY booking_cost DESC", conn)
# Can't use HAVING without GROUP BY in SQLITE so re-using CASE instead of HAVING booking_cost > 30

,name,full_name,booking_cost
0,Massage Room 2,GUESTGUEST,320.0
1,Massage Room 1,GUESTGUEST,160.0
2,Massage Room 1,GUESTGUEST,160.0
3,Massage Room 1,GUESTGUEST,160.0
4,Tennis Court 2,GUESTGUEST,150.0
5,Tennis Court 1,GUESTGUEST,75.0
6,Tennis Court 1,GUESTGUEST,75.0
7,Tennis Court 2,GUESTGUEST,75.0
8,Squash Court,GUESTGUEST,70.0
9,Massage Room 1,FarrellJemima,39.6


In [58]:
#Q9: This time, produce the same result as in Q8, but using a subquery.

pd.read_sql_query("SELECT name, full_name, booking_cost "
            "FROM "
                "(SELECT B.starttime, F.name, (M.surname||M.firstname) AS full_name, B.bookid, "
                "CASE WHEN M.memid =0 THEN (F.guestcost * B.slots) ELSE (F.membercost * B.slots) END AS booking_cost "
                 "FROM Bookings B "
                 "INNER JOIN Facilities F ON F.facid = B.facid "
                 "INNER JOIN Members M ON M.memid = B.memid "
                ") sub "
        "WHERE starttime LIKE '2012-09-14%' AND booking_cost > 30 "
        "ORDER BY booking_cost DESC", conn)

,name,full_name,booking_cost
0,Massage Room 2,GUESTGUEST,320.0
1,Massage Room 1,GUESTGUEST,160.0
2,Massage Room 1,GUESTGUEST,160.0
3,Massage Room 1,GUESTGUEST,160.0
4,Tennis Court 2,GUESTGUEST,150.0
5,Tennis Court 1,GUESTGUEST,75.0
6,Tennis Court 1,GUESTGUEST,75.0
7,Tennis Court 2,GUESTGUEST,75.0
8,Squash Court,GUESTGUEST,70.0
9,Massage Room 1,FarrellJemima,39.6


In [60]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members!

pd.read_sql_query("SELECT total_slots_table.facid, total_guest_slots, total_memb_slots, "
          "((F.membercost * total_memb_slots) + (F.guestcost * total_guest_slots)) AS rev_per_fac "
                "FROM "
                "(SELECT Bookings.facid, SUM(slots) AS total_guest_slots, total_memb_slots "
                 "FROM Bookings "
                 "INNER JOIN "
                     "(SELECT Bookings.facid, SUM(slots) AS total_memb_slots "
                     "FROM Bookings "
                     "WHERE memid != 0 "
                     "GROUP BY Bookings.facid "
                    ") memb_slot_table "
                 "ON memb_slot_table.facid = Bookings.facid "
                 "WHERE memid = 0 "
                 "GROUP BY Bookings.facid "
                ") total_slots_table "
        "INNER JOIN Facilities F ON F.facid = total_slots_table.facid "
        "WHERE ((F.membercost * total_memb_slots) + (F.guestcost * total_guest_slots)) < 1000 "
        "ORDER BY rev_per_fac", conn)

,facid,total_guest_slots,total_memb_slots,rev_per_fac
0,3,36,794,180
1,7,48,860,240
2,8,54,856,270
